In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Stops auto-scrolling so entire output is visible: see https://stackoverflow.com/a/41646403

In [ ]:
# Default parameter values. They will be overwritten by papermill notebook parameters.
# This cell must carry the tag "parameters" in its metadata.
from pathlib import Path
import pickle
import codecs

innereye_path = Path.cwd().parent.parent.parent
train_metrics_csv = ""
val_metrics_csv = ""
test_metrics_csv = ""
number_best_and_worst_performing = 20
config= ""

In [ ]:
import sys
print(f"Adding to path: {innereye_path}")
if str(innereye_path) not in sys.path:
    sys.path.append(str(innereye_path))

%matplotlib inline
import matplotlib.pyplot as plt

config = pickle.loads(codecs.decode(config.encode(), "base64"))

from InnerEye.ML.reports.notebook_report import print_header
from InnerEye.ML.reports.classification_report import print_metrics, \
ReportedMetrics, get_metric, get_labels_and_predictions
from InnerEye.ML.reports.classification_multilabel_report import get_unique_prediction_target_combinations, \
get_labels_and_predictions_for_prediction_target_set

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (20, 10)

#convert params to Path
train_metrics_csv = Path(train_metrics_csv)
val_metrics_csv = Path(val_metrics_csv)
test_metrics_csv = Path(test_metrics_csv)

In [ ]:
unique_prediction_target_combinations = get_unique_prediction_target_combinations(config)
all_prediction_target_combinations = list(set([(l,) for l in config.class_names]) | set(map(tuple, unique_prediction_target_combinations)))

# Train Metrics (for label combinations)

In [ ]:
if train_metrics_csv.is_file():
    thresholds_per_prediction_target = []
    for label in config.class_names:
        train_metrics = get_labels_and_predictions(train_metrics_csv, label)
        thresholds_per_prediction_target.append(get_metric(val_labels_and_predictions=train_metrics,
                                                test_labels_and_predictions=train_metrics,
                                                metric=ReportedMetrics.OptimalThreshold))

    for labels in all_prediction_target_combinations:
        print_header(f"Class {'|'.join(labels) or 'Negative'}", level=3)
        train_metrics = get_labels_and_predictions_for_prediction_target_set(train_metrics_csv, labels, config.class_names, thresholds_per_prediction_target)
        print_metrics(val_labels_and_predictions=train_metrics, test_labels_and_predictions=train_metrics, is_thresholded=True)

# Validation Metrics (for label combinations)

In [ ]:
if val_metrics_csv.is_file():
    thresholds_per_prediction_target = []
    for label in config.class_names:
        val_metrics = get_labels_and_predictions(val_metrics_csv, label)
        thresholds_per_prediction_target.append(get_metric(val_labels_and_predictions=val_metrics,
                                                test_labels_and_predictions=val_metrics,
                                                metric=ReportedMetrics.OptimalThreshold))

    for labels in all_prediction_target_combinations:
        print_header(f"Class {'|'.join(labels) or 'Negative'}", level=3)
        val_metrics = get_labels_and_predictions_for_prediction_target_set(val_metrics_csv, labels, config.class_names, thresholds_per_prediction_target)
        print_metrics(val_labels_and_predictions=val_metrics, test_labels_and_predictions=val_metrics, is_thresholded=True)

# Test Metrics (for label combinations)

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    thresholds_per_prediction_target = []
    for label in config.class_names:
        val_metrics = get_labels_and_predictions(val_metrics_csv, label)
        test_metrics = get_labels_and_predictions(test_metrics_csv, label)
        thresholds_per_prediction_target.append(get_metric(val_labels_and_predictions=val_metrics,
                                                test_labels_and_predictions=test_metrics,
                                                metric=ReportedMetrics.OptimalThreshold))

    for labels in all_prediction_target_combinations:
        print_header(f"Class {'|'.join(labels) or 'Negative'}", level=3)
        val_metrics = get_labels_and_predictions_for_prediction_target_set(val_metrics_csv, labels, config.class_names, thresholds_per_prediction_target)
        test_metrics = get_labels_and_predictions_for_prediction_target_set(test_metrics_csv, labels, config.class_names, thresholds_per_prediction_target)
        print_metrics(val_labels_and_predictions=val_metrics, test_labels_and_predictions=test_metrics, is_thresholded=True)